## CNN_LSTM Base
- to capture spatial (CNN) -temporal (LSTM) information
- often used in NEXT FRAME VIDEO PREDICTION problem in vision

### Colab Link
- Run it with GPU: https://colab.research.google.com/drive/1mNi_gSTWDSto7EWnyh4ZVuQA3VfN56Hm#scrollTo=zVGtIJiqnJLB


### Sources

https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9063513

https://keras.io/examples/vision/conv_lstm/

In [ ]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from utils import df_to_xarray,read_xarray,plot_image,preprocess_image,create_shifted_frames

sys.path.insert(0, '../src')
#!module load cuda11.0/toolkit cuda11.0/blas cudnn8.0-cuda11.0
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

sys.path.insert(0, '../src')
# Reading Data

dir_name="../data/data1"
val_dir_name="../data/data2"

chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)

In [ ]:
# Preprocessing the Data
chl_images=preprocess_image(chl.Chl.data)
mld_images=preprocess_image(mld.MLD.data)
sss_images=preprocess_image(sss.SSS.data)
sst_images=preprocess_image(sst.SST.data)
xco2_images=preprocess_image(xco2.XCO2.data,xco2=True)
pco2_images=preprocess_image(pco2.pCO2.data,pco2=True)

x_train, y_train = create_shifted_frames(train_data)

print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))

In [ ]:
inp = layers.Input(shape=(None, *x_train.shape[2:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.Conv3D(
    filters=1, kernel_size=(3, 3, 3), activation="relu", padding="same"
)(x)

# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.compile(
    loss="mean_squared_error", optimizer=keras.optimizers.Adam(),
)

In [ ]:
model.summary()

In [ ]:
from functools import partial

INPUT_SHAPE=X[0].shape
OUTPUT_SHAPE=pco2_images[0].shape

In [ ]:
model_path="../models/base_CNN_LSTM.h5"

early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]
# Define modifiable training hyperparameters.
epochs = 20
batch_size = 4

# Fit the model to the training data.
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_train, y_train),
    callbacks=callbacks,
)